In [2]:
import requests
import hashlib
import time
import pandas as pd
from pandas import json_normalize

secret_token = '696a6f47212d93f88cef71238cfb3d1cd357905b3e34eabd2bbc49124553c129'

# Generate the Unix timestamp
timestamp = int(time.time())

# Construct the query string
query_string = f'time={timestamp}'

# Generate the hash
hash_string = secret_token + str(timestamp)
hash_value = hashlib.sha512(hash_string.encode()).hexdigest()

# Construct the URL with the query string and hash
url = f'https://api.fabdb.net/cards?{query_string}&hash={hash_value}'

parameters = {
    "page": 1,
    "per_page": 100
}

total_pages = 19

# Set the request headers
headers = {
    'Authorization': f'Bearer {secret_token}',
    'Accept': 'application/json'
}


In [3]:
data_records = []

for page in range(1, total_pages + 1):
    parameters['page'] = page
    
    response = requests.get(url, params=parameters)

    # process response
    data = response.json()

    # Extract the relevant data records from the response
    data_records.extend(data['data'])
pd.set_option('display.max_columns', None)
# Normalize the data before storing it in the dataframe
df2 = pd.json_normalize(data_records)

In [4]:
df2.head(4)

,identifier,name,legality,keywords,text,rarity,image,printings,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 863, 'language': 'en', 'name': 'Eye of...",3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 864, 'language': 'en', 'name': 'Dash, ...",NaN,40.0,4.0,NaN,NaN,NaN,NaN
2,dash,Dash,[],"[mechanologist, hero, young]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 865, 'language': 'en', 'name': 'Dash',...",NaN,20.0,4.0,NaN,NaN,NaN,NaN
3,teklo-plasma-pistol,Teklo Plasma Pistol,[],"[mechanologist, weapon, pistol, 2h]",**Action** - Remove a steam counter from Teklo...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 866, 'language': 'en', 'name': 'Teklo ...",NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [5]:
# Explode the 'printings' column to expand the nested list
df_printings = df2.explode('printings')

# Expand the 'printings' column into separate columns
df_printings = pd.concat([df_printings['printings'].apply(pd.Series), df_printings], axis=1)

# Merge the expanded data with the original DataFrame
df_merged = pd.concat([df2.drop('printings', axis=1), df_printings], axis=1)

# Drop the original 'printings' column
df_merged.drop('printings', axis=1, inplace=True)
df_merged

,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats,id,language,name,text,flavour,sku,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...","{'sku': 'ARC000-CF', 'finish': 'cold', 'set': ...",Arcane Rising,F,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...","{'sku': 'U-ARC000-RF', 'finish': 'rainbow', 's...",Arcane Rising Unlimited,F,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,NaN,40.0,4.0,NaN,NaN,NaN,NaN,864,en,"Dash, Inventor Extraordinaire",You may start the game with a Mechanologist it...,,"{'sku': 'ARC001', 'finish': 'regular', 'set': ...",Arcane Rising,T,Regular,{},https://fabdb2.imgix.net/cards/printings/ARC00...,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,NaN,40.0,4.0,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,NaN,40.0,4.0,NaN,NaN,NaN,NaN,1282,en,"Dash, Inventor Extraordinaire",You may start the game with a Mechanologist it...,,"{'sku': 'U-ARC001', 'finish': 'regular', 'set'...",Arcane Rising Unlimited,T,Regular,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,NaN,40.0,4.0,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,NaN,40.0,4.0,NaN,NaN,NaN,NaN,1885,en,"Dash, Inventor Extraordinaire",You may start the game with a Mechanologist it...,,"{'sku': 'CRU098', 'finish': 'regular', 'set': ...",Crucible of War,C,Regular,{},https://fabdb2.imgix.net/cards/printings/CRU09...,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,NaN,40.0,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,phoenix-form,Phoenix Form,[],"[draconic, ninja, action, attack]","If you control 1 or more Phoenix Flames, Phoen...",M,https://fabdb2.imgix.net/cards/printings/UPR04...,1.0,NaN,NaN,3.0,3.0,0.0,NaN,12118,en,Phoenix